### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [11]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [14]:
ratings = pd.read_csv('../data/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [15]:
movie_info = pd.read_csv('../data/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [16]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [17]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [18]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [19]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [11]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [12]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [13]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [14]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [15]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [5]:
# imports
import time
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from typing import Tuple, List
from lightfm.datasets import fetch_movielens
from sklearn.metrics.pairwise import cosine_similarity
from scipy.special import expit

In [21]:
# util functions
def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        return time.time() - start

    return wrapper

def calc_auc(model, R):
    R = R.toarray()
    auc = 0
    for u_id, u in enumerate(R):
        if np.count_nonzero(u != 0) == 0:
            continue

        pos_mask = u > 0
        neg_mask = ~u

        preds = model.I @ model.U[u_id]
        comp_matrix = preds[pos_mask].reshape(-1, 1) > preds[neg_mask]
        u_auc = np.count_nonzero(comp_matrix) / (comp_matrix.shape[0] * comp_matrix.shape[1])

        auc += u_auc

    return auc / R.shape[0]


def test_model(model, implicit_ratings, movie_info):
    get_similars = lambda item_id, model: [
        movie_info[movie_info["movie_id"] == x]["name"].to_string() for x in model.similar_items(item_id)
    ]
    print("Similars:")
    for r in get_similars(1, model): 
        print(r)

    get_user_history = lambda user_id, implicit_ratings: [
        movie_info[movie_info["movie_id"] == x]["name"].to_string()
        for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]
    ]
    
    print("User's history:")
    for f in get_user_history(4, implicit_ratings):
        print(f)

    get_recommendations = lambda user_id, model: [
        movie_info[movie_info["movie_id"] == x]["name"].to_string() for x in model.recommend(user_id)
    ]
    
    print("Recommendations:")
    for r in get_recommendations(4, model): 
        print(r)

In [7]:
class MatrixFactorization:
    def __init__(self, lr: float = 0.01, l2: float = 0.01):
        self.lr = lr
        self.l2 = l2

        self.I = np.ndarray
        self.U = np.ndarray

    def _calc_metrics(self, R) -> Tuple[float, float]:
        R = R.toarray()
        non_zero_mask = R != 0
        scores = self.U @ self.I.T
        mse = np.sum((R[non_zero_mask] - scores[non_zero_mask]) ** 2)
        rmse = np.sqrt(mse / np.count_nonzero(non_zero_mask))
        reg = self.l2 * (np.sum(np.linalg.norm(self.I, axis=1) ** 2) + np.sum(np.linalg.norm(self.U, axis=1) ** 2))

        return mse + reg, rmse

    def _print_info(
        self,
        R,
        step,
        iteration_time,
        verbose,
        use_auc=False,
        print_every=1,
    ):
        if verbose and step % print_every == 0:
            loss, rmse = self._calc_metrics(R)
            res_str = f"Iteration: {step + 1}; time: {iteration_time:.1f}; loss: {loss:.2f}; rmse: {rmse: .5f}"
            if use_auc:
                res_str += f"; AUC: {calc_auc(self, R): .3f}"
            print(res_str)

    def _initialize_matrix(self, n_users, n_items, hidden_dim):
        self.U = np.random.uniform(0, 1 / np.sqrt(hidden_dim), size=(n_users, hidden_dim))
        self.I = np.random.normal(0, 1 / np.sqrt(hidden_dim), size=(n_items, hidden_dim))

    def recommend(self, user_id: int, n: int = 10):
        scores = self.I @ self.U[user_id]
        return np.argsort(-scores)[:n]

    def similar_items(self, item_id: int, n: int = 10):
        scores = cosine_similarity(self.I)
        return np.argsort(-scores[item_id])[:n]

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [8]:
class SVD(MatrixFactorization):
    def fit(self, R, hidden_dim: int = 10, n_iters: int = 10, verbose: bool = True):
        n_users, n_items = R.shape
        self._initialize_matrix(n_users, n_items, hidden_dim)

        samples = [(i, j, k) for i, j, k in zip(R.row, R.col, R.data)]

        for step in range(n_iters):
            iteration_time = self._update_matrix(samples)

            self._print_info(R, step, iteration_time, verbose)

    @time_it
    def _update_matrix(self, samples: List[Tuple[int, int, int]]):
        U, I = self.U, self.I
        lr, l2 = self.lr, self.l2

        shuffled_samples = np.random.permutation(samples)
        for u_id, i_id, r in shuffled_samples:
            error = U[u_id] @ I[i_id] - r
            U[u_id] = U[u_id] - lr * 2 * (error * I[i_id] + l2 * U[u_id])
            I[i_id] = I[i_id] - lr * 2 * (error * U[u_id] + l2 * I[i_id])

In [22]:
user_item_exp = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

svd = SVD(lr=0.01, l2=0.01)
svd.fit(user_item_exp, n_iters=2, hidden_dim=64)

test_model(svd, implicit_ratings, movie_info)

Iteration: 1; time: 16.2; loss: 840340.09; rmse:  0.91642
Iteration: 2; time: 16.3; loss: 687646.32; rmse:  0.82892
Similars:
0    Toy Story (1995)
1205    Grand Day Out, A (1992)
1449    Love Jones (1997)
3535    Gypsy (1962)
2009    Jungle Book, The (1967)
3045    Toy Story 2 (1999)
2011    Lady and the Tramp (1955)
3282    Two Thousand Maniacs! (1964)
935    My Man Godfrey (1936)
3175    Goodbye Girl, The (1977)
User's history:
3399    Hustler, The (1961)
2882    Fistful of Dollars, A (1964)
1196    Alien (1979)
1023    Die Hard (1988)
257    Star Wars: Episode IV - A New Hope (1977)
1959    Saving Private Ryan (1998)
476    Jurassic Park (1993)
1180    Raiders of the Lost Ark (1981)
1885    Rocky (1976)
1081    E.T. the Extra-Terrestrial (1982)
3349    Thelma & Louise (1991)
3633    Mad Max (1979)
2297    King Kong (1933)
1366    Jaws (1975)
1183    Good, The Bad and The Ugly, The (1966)
2623    Run Lola Run (Lola rennt) (1998)
2878    Goldfinger (1964)
1220    Terminator, The (198

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [24]:
class ALS(MatrixFactorization):
    def fit(self, R, hidden_dim: int = 10, n_iters: int = 10, verbose: bool = True):
        n_users, n_items = R.shape
        self._initialize_matrix(n_users, n_items, hidden_dim)

        for step in range(n_iters):
            iteration_time = self._update_matrix(self.U, self.I, R, hidden_dim, True) + self._update_matrix(
                self.I, self.U, R, hidden_dim, False
            )

            self._print_info(R, step, iteration_time, verbose)

    @time_it
    def _update_matrix(self, X, Y, R, hidden_dim, item: bool):
        n_objects = X.shape[0]

        # updating users
        for i in range(n_objects):
            if item:
                r = R[i].toarray().flatten()
            else:
                r = R[:, i].toarray().flatten()

            non_zero_ids_mask = r != 0
            if np.count_nonzero(non_zero_ids_mask) == 0:
                continue

            ys = Y[non_zero_ids_mask]

            # sum of outer products of y
            outer_prods = Y.T @ Y
            inv = np.eye(hidden_dim) * self.l2 + outer_prods
            inv = np.linalg.inv(inv)

            # sum of r * y
            r_part = np.sum(r[non_zero_ids_mask].reshape(-1, 1) * ys, axis=0)

            X[i] = inv @ r_part

In [25]:
als = ALS(l2=1)
als.fit(user_item_csr, n_iters=5, hidden_dim=64)

Iteration: 1; time: 24.5; loss: 355073.01; rmse:  0.76690
Iteration: 2; time: 23.5; loss: 277206.41; rmse:  0.67375
Iteration: 3; time: 25.8; loss: 265931.31; rmse:  0.66064
Iteration: 4; time: 28.0; loss: 261219.91; rmse:  0.65569
Iteration: 5; time: 19.9; loss: 258550.29; rmse:  0.65312


In [26]:
test_model(als, implicit_ratings, movie_info)

Similars:
0    Toy Story (1995)
3045    Toy Story 2 (1999)
2286    Bug's Life, A (1998)
584    Aladdin (1992)
360    Lion King, The (1994)
2252    Pleasantville (1998)
2618    Tarzan (1999)
33    Babe (1995)
1838    Mulan (1998)
1526    Hercules (1997)
User's history:
3399    Hustler, The (1961)
2882    Fistful of Dollars, A (1964)
1196    Alien (1979)
1023    Die Hard (1988)
257    Star Wars: Episode IV - A New Hope (1977)
1959    Saving Private Ryan (1998)
476    Jurassic Park (1993)
1180    Raiders of the Lost Ark (1981)
1885    Rocky (1976)
1081    E.T. the Extra-Terrestrial (1982)
3349    Thelma & Louise (1991)
3633    Mad Max (1979)
2297    King Kong (1933)
1366    Jaws (1975)
1183    Good, The Bad and The Ugly, The (1966)
2623    Run Lola Run (Lola rennt) (1998)
2878    Goldfinger (1964)
1220    Terminator, The (1984)
Recommendations:
1180    Raiders of the Lost Ark (1981)
257    Star Wars: Episode IV - A New Hope (1977)
1366    Jaws (1975)
1196    Alien (1979)
1081    E.T. the 

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [27]:
class BPR(MatrixFactorization):
    def fit(self, R, hidden_dim: int = 10, n_iters: int = 10, batch_size: int = 32, verbose: bool = True):
        n_users, n_items = R.shape
        self._initialize_matrix(n_users, n_items, hidden_dim)

        indptr = R.indptr
        indices = R.indices
        n_users, n_items = R.shape

        for step in range(n_iters):
            users, items_pos, items_neg = self._sample(n_users, n_items, batch_size, indices, indptr)
            iteration_time = self._grad_step(users, items_pos, items_neg)

            self._print_info(R, step, iteration_time, verbose, True, 100)

    @time_it
    def _grad_step(self, u, ii, ij):
        user_u = self.U[u]
        item_i = self.I[ii]
        item_j = self.I[ij]

        r_uij = np.sum(user_u * (item_i - item_j), axis=1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))

        hidden_dim = self.I.shape[1]
        sigmoid_tiled = np.tile(sigmoid, (hidden_dim, 1)).T

        grad_u = sigmoid_tiled * (item_j - item_i) + self.l2 * user_u
        grad_i = sigmoid_tiled * -user_u + self.l2 * item_i
        grad_j = sigmoid_tiled * user_u + self.l2 * item_j
        self.U[u] -= self.lr * grad_u
        self.I[ii] -= self.lr * grad_i
        self.I[ij] -= self.lr * grad_j

    def _sample(self, n_users, n_items, batch_size, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(batch_size, dtype=np.int)
        sampled_neg_items = np.zeros(batch_size, dtype=np.int)
        sampled_users = np.random.choice(n_users, size=batch_size, replace=False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user] : indptr[user + 1]]

            if len(pos_items) == 0:
                continue

            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items


In [28]:
bpr = BPR(lr=1e-2, l2=0.01)
bpr.fit(user_item_csr, n_iters=8000, hidden_dim=64, batch_size=user_item_csr.shape[0])

Iteration: 1; time: 0.0; loss: 579000.48; rmse:  1.00318; AUC:  0.510
Iteration: 101; time: 0.0; loss: 556679.65; rmse:  0.98365; AUC:  0.600
Iteration: 201; time: 0.0; loss: 536216.71; rmse:  0.96539; AUC:  0.682
Iteration: 301; time: 0.0; loss: 520711.75; rmse:  0.95131; AUC:  0.694
Iteration: 401; time: 0.0; loss: 524601.40; rmse:  0.95484; AUC:  0.725
Iteration: 501; time: 0.0; loss: 568505.29; rmse:  0.99398; AUC:  0.686
Iteration: 601; time: 0.0; loss: 669776.66; rmse:  1.07888; AUC:  0.689
Iteration: 701; time: 0.0; loss: 844222.18; rmse:  1.21127; AUC:  0.806
Iteration: 801; time: 0.0; loss: 1114748.90; rmse:  1.39190; AUC:  0.816
Iteration: 901; time: 0.0; loss: 1496858.67; rmse:  1.61293; AUC:  0.818
Iteration: 1001; time: 0.0; loss: 2015487.37; rmse:  1.87163; AUC:  0.775
Iteration: 1101; time: 0.0; loss: 2689173.34; rmse:  2.16195; AUC:  0.798
Iteration: 1201; time: 0.0; loss: 3535363.88; rmse:  2.47889; AUC:  0.824
Iteration: 1301; time: 0.0; loss: 4563936.52; rmse:  2.816

In [29]:
test_model(bpr, implicit_ratings, movie_info)

Similars:
0    Toy Story (1995)
1446    Jungle2Jungle (a.k.a. Jungle 2 Jungle) (1997)
565    Little Big League (1994)
3045    Toy Story 2 (1999)
370    Richie Rich (1994)
2807    Thumbelina (1994)
584    Aladdin (1992)
3327    Muppet Movie, The (1979)
1967    Blank Check (1994)
2759    Dudley Do-Right (1999)
User's history:
3399    Hustler, The (1961)
2882    Fistful of Dollars, A (1964)
1196    Alien (1979)
1023    Die Hard (1988)
257    Star Wars: Episode IV - A New Hope (1977)
1959    Saving Private Ryan (1998)
476    Jurassic Park (1993)
1180    Raiders of the Lost Ark (1981)
1885    Rocky (1976)
1081    E.T. the Extra-Terrestrial (1982)
3349    Thelma & Louise (1991)
3633    Mad Max (1979)
2297    King Kong (1933)
1366    Jaws (1975)
1183    Good, The Bad and The Ugly, The (1966)
2623    Run Lola Run (Lola rennt) (1998)
2878    Goldfinger (1964)
1220    Terminator, The (1984)
Recommendations:
1220    Terminator, The (1984)
1183    Good, The Bad and The Ugly, The (1966)
257    Star

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных